
## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [**infinite monkeys typing for an infinite amount of time**](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of William Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

We will focus specifically on Shakespeare's Sonnets in order to improve our model's ability to learn from the data.

In [1]:
import random
import sys
import os

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LambdaCallback

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional
from tensorflow.keras.layers import LSTM

%matplotlib inline

# a custom data prep class that we'll be using 
from data_cleaning_toolkit_class import data_cleaning_toolkit

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


### Use request to pull data from a URL

[**Read through the request documentation**](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) in order to learn how to download the Shakespeare Sonnets from the Gutenberg website. 

**Protip:** Do not over think it.

In [2]:
# download all of Shakespears Sonnets from the Project Gutenberg website

# here's the link for the sonnets
url_shakespeare_sonnets = "https://www.gutenberg.org/cache/epub/1041/pg1041.txt"

# use request and the url to download all of the sonnets - save the result to `r`

r = requests.get(url_shakespeare_sonnets)

In [3]:
dir(r)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [4]:
# move the downloaded text out of the request object - save the result to `raw_text_data`
# hint: take at look at the attributes of `r`

raw_text_data = r.text

In [5]:
# check the data type of `raw_text_data`
type(raw_text_data)

str

### Data Cleaning

In [6]:
# as usual, we are tasked with cleaning up messy data
# Question: Do you see any characters that we could use to split up the text?
raw_text_data[:3000]

"\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Shakespeare's Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nPosting Date: April 7, 2014 [EBook #1041]\r\nRelease Date: September, 1997\r\nLast Updated: March 10, 2010\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***\r\n\r\n\r\n\r\n\r\nProduced by Joseph S. Miller and Embry-Riddle Aeronautical\r\nUniversity Library. HTML version by Al Haines.\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\n  I\r\n\r\n  From fairest creatures we desire increase,\r\n  That thereby beauty's rose might never die,\r\n  But as the riper

In [7]:
dctk = data_cleaning_toolkit()

In [8]:
text = dctk.clean_data(raw_text_data)

In [9]:
split_data = text.split("\r")

In [10]:
# split the text into lines and save the result to `split_data`
# YOUR CODE HERE
# dir(data_cleaning_toolkit())
# type(data_cleaning_toolkit())
# for delimiter in delimiters:
#     new_words = []
#     for word in words:
#         new_words += word.split(delimiter)
#     split_data = new_words

# split_data = words(raw_text_data)

In [11]:
# we need to drop all the boilder plate text (i.e. titles and descriptions) as well as white spaces
# so that we are left with only the sonnets themselves 
split_data[:20] 

['the project gutenberg ebook of shakespeares sonnets by william shakespeare',
 ' ',
 ' this ebook is for the use of anyone anywhere at no cost and with',
 ' almost no restrictions whatsoeveryou may copy it give it away or',
 ' reuse it under the terms of the project gutenberg license included',
 ' with this ebook or online at wwwgutenbergorg',
 ' ',
 ' ',
 ' title shakespeares sonnets',
 ' ',
 ' author william shakespeare',
 ' ',
 ' posting date aprilebook ',
 ' release date september ',
 ' last updated march',
 ' ',
 ' language english',
 ' ',
 ' ',
 'start of this project gutenberg ebook shakespeares sonnets ']

**Use list index slicing in order to remove the titles and descriptions so we are only left with the sonnets.**


In [12]:
# sonnets exists between these indicies 
# titles and descriptions exist outside of these indicies

# use index slicing to isolate the sonnet lines - save the result to `sonnets`

sonnets = split_data[35:]

In [13]:
# notice how all non-sonnet lines have far less characters than the actual sonnet lines?
# well, let's use that observation to filter out all the non-sonnet lines
sonnets[200:240]

['when i behold the violet past prime',
 'and sable curls all silvered oer with white',
 'when lofty trees i see barren of leaves',
 'which erst from heat did canopy the herd',
 'and summers green all girded up in sheaves',
 'borne on the bier with white and bristly beard',
 'then of thy beauty do i question make',
 'that thou among the wastes of time must go',
 'since sweets and beauties do themselves forsake',
 'and die as fast as they see others grow',
 'and nothing gainst times scythe can make defence',
 'save breed to brave him when he takes thee hence',
 ' ',
 'xiii',
 ' ',
 'o that you were your self but love you are',
 'no longer yours than you your self here live',
 'against this coming end you should prepare',
 'and your sweet semblance to some other give',
 'so should that beauty which you hold in lease',
 'find no determination then you were',
 'yourself again after yourselfs decease',
 'when your sweet issue your sweet form should bear',
 'who lets so fair a house fall to 

In [14]:
# any string with less than n_chars characters will be filtered out - save results to `filtered_sonnets`

# filtered_sonnets = list(set(sonnets))

In [15]:
# filtered_sonnets

In [16]:
# ok - much better!
# but we still need to remove all the punctuation and case normalize the text
# filtered_sonnets

### Use custom data cleaning tool 

Use one of the methods in `data_cleaning_toolkit` to clean your data.

There is an example of this in the guided project.

In [17]:
# instantiate the data_cleaning_toolkit class - save result to `dctk`

# dctk = data_cleaning_toolkit()

In [18]:
# use data_cleaning_toolkit to remove punctuation and to case normalize - save results to `clean_sonnets`

# dir(dctk)

In [19]:
# dctk.clean_data(split_data)

In [20]:
clean_sonnets = list(set(sonnets))

In [21]:
# much better!
len(clean_sonnets)

2607

In [22]:
len(clean_sonnets[5])

41

### Use your data tool to create character sequences 

We'll need the `create_char_sequenes` method for this task. However this method requires a parameter call `maxlen` which is responsible for setting the maximum sequence length. 

So what would be a good sequence length, exactly? 

In order to answer that question, let's do some statistics! 

In [23]:
def calc_stats(corpus):
    """
    Calculates statisics on the length of every line in the sonnets
    """
    
    # write a list comprehension that calculates each sonnets line length - save the results to `doc_lens` 

    # use numpy to calcualte and return the mean, median, std, max, min of the doc lens - all in one line of code
    
    temp = [len(ele) for ele in corpus]
    res = 0 if len(temp) == 1 else temp
    
    return np.mean(res), np.median(res), np.std(res), np.max(res), np.min(res)

In [24]:
# sonnet line length statistics 
mean, med, std, max_, min_ = calc_stats(clean_sonnets)
mean, med, std, max_, min_ 

(40.783659378596084, 41.0, 12.622578632237355, 73, 1)

In [25]:
dctk.create_char_sequenes(clean_sonnets, maxlen=41)

sequences:  21778


In [26]:
dir(dctk)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'char_int',
 'chars',
 'clean_data',
 'create_X_and_Y',
 'create_char_sequenes',
 'int_char',
 'maxlen',
 'n_features',
 'next_char',
 'sequences']

In [27]:
# using the results of the sonnet line length statistics, use your judgement and select a value for maxlen
# use .create_char_sequences() to create sequences



Take a look at the `data_cleaning_toolkit_class.py` file. 

In the first 4 lines of code in the `create_char_sequences` method, class attributes `n_features` and `unique_chars` are created. Let's call them in the cells below.

In [28]:
# number of input features for our LSTM model
dctk.n_features

27

In [29]:
# unique charactes that appear in our sonnets 
dctk.chars

['r',
 'g',
 'y',
 ' ',
 'v',
 'f',
 'b',
 't',
 's',
 'j',
 'm',
 'l',
 'a',
 'z',
 'p',
 'i',
 'w',
 'q',
 'o',
 'u',
 'h',
 'x',
 'd',
 'k',
 'c',
 'e',
 'n']

## Time for Questions 

----
**Question 1:** 

Why are the `number of unique characters` (i.e. **dctk.unique_chars**) and the `number of model input features` (i.e. **dctk.n_features**) the same?

**Hint:** The model that we will shortly be building here is very similar to the text generation model that we built in the guided project.

**Answer 1:**

Each letter claims its own feature as each word changes its inherent sequence based upon the letters used.


**Question 2:**

Take a look at the print out of `dctk.unique_chars` one more time. Notice that there is a white space. 

Why is it desirable to have a white space as a possible character to predict?

**Answer 2:**

If a whitespace is eschewed, the words will just become jumbled together.

----

### Use our data tool to create X and Y splits

You'll need the `create_X_and_Y` method for this task. 

In [30]:
# TODO: provide a walk through of data_cleaning_toolkit with unit tests that check for understanding 
X, y = dctk.create_X_and_Y()

![](https://miro.medium.com/max/891/0*jGB1CGQ9HdeUwlgB)

In [31]:
# notice that our input matrix isn't actually a matrix - it's a rank 3 tensor
X.shape

(21778, 41, 27)

In $X$.shape we see three numbers (*n1*, *n2*, *n3*). What do these numbers mean?

Well, *n1* tells us the number of samples that we have. But what about the other two?

In [32]:
# first index returns a signle sample, which we can see is a sequence 
first_sample_index = 0 
X[first_sample_index]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

Notice that each sequence (i.e. $X[i]$ where $i$ is some index value) is `maxlen` long and has `dctk.n_features` number of features. Let's try to better understand this shape. 

In [33]:
# each sequence is maxlen long and has dctk.n_features number of features
X[first_sample_index].shape

(41, 27)

**Each row corresponds to a character vector** and there are `maxlen` number of character vectors. 

**Each column corresponds to a unique character** and there are `dctk.n_features` number of features. 


In [34]:
# let's index for a single character vector 
first_char_vect_index = 0
X[first_sample_index][first_char_vect_index]

array([False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

Notice that there is a single `TRUE` value and all the rest of the values are `FALSE`. 

This is a one-hot encoding for which character appears at each index within a sequence. Specifically, the cell above is looking at the first character in the sequence.

Only a single character can appear as the first character in a sequence, so there will necessarily be a single `TRUE` value and the rest will be `FALSE`. 

Let's say that `TRUE` appears in the $ith$ index; by  $ith$ index we simply mean some index in the general case. How can we find out which character that actually corresponds to?

To answer this question, we need to use the character-to-integer look up dictionaries. 

In [35]:
# take a look at the index to character dictionary
# if a TRUE appears in the 0th index of a character vector,
# then we know that whatever char you see below next to the 0th key 
# is the character that that character vector is endcoding for
dctk.int_char

{0: 'r',
 1: 'g',
 2: 'y',
 3: ' ',
 4: 'v',
 5: 'f',
 6: 'b',
 7: 't',
 8: 's',
 9: 'j',
 10: 'm',
 11: 'l',
 12: 'a',
 13: 'z',
 14: 'p',
 15: 'i',
 16: 'w',
 17: 'q',
 18: 'o',
 19: 'u',
 20: 'h',
 21: 'x',
 22: 'd',
 23: 'k',
 24: 'c',
 25: 'e',
 26: 'n'}

In [36]:
# let's look at an example to tie it all together

seq_len_counter = 0

# index for a single sample 
for seq_of_char_vects in X[first_sample_index]:
    
    # get index with max value, which will be the one TRUE value 
    index_with_TRUE_val = np.argmax(seq_of_char_vects)
    
    print (dctk.int_char[index_with_TRUE_val])
    
    seq_len_counter+=1
    
print ("Sequence length: {}".format(seq_len_counter))

a
n
d
 
s
o
 
o
f
 
y
o
u
 
b
e
a
u
t
e
o
u
s
 
a
n
d
 
l
o
v
e
l
y
 
y
o
u
t
h
 
Sequence length: 41


## Time for Questions 

----
**Question 1:** 

In your own words, how would you describe the numbers from the shape print out of `X.shape` to a fellow classmate?


In [37]:
X.shape

(21778, 41, 27)

**Answer 1:**

The first number is a binary classification of sequences; the second describes the words we have in each line; the third number describes the possible characters within each word.

----


### Build a Text Generation model

Now that we have prepped our data (and understood that process) let's finally build out our character generation model, similar to what we did in the guided project.

In [38]:
def sample(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array
    """
    # convert preds to array 
    preds = np.asarray(preds).astype('float64')
    # scale values 
    preds = np.log(preds) / temperature
    # exponentiate values
    exp_preds = np.exp(preds)
    # this equation should look familar to you (hint: it's an activation function)
    preds = exp_preds / np.sum(exp_preds)
    # Draw samples from a multinomial distribution
    probas = np.random.multinomial(1, preds, 1)
    # return the index that corresponds to the max probability 
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    """"
    Function invoked at end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # randomly pick a starting index 
    # will be used to take a random sequence of chars from `text`
    start_index = random.randint(0, len(text) - dctk.maxlen - 1)
    
    # this is our seed string (i.e. input seqeunce into the model)
    generated = ''

    # start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
    sentence = text[start_index: start_index + dctk.maxlen]

    # add to generated
    generated += sentence

    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # use model to predict what the next 40 chars should be that follow the seed string
    for i in range(40):

        # shape of a single sample in a rank 3 tensor 
        x_dims = (1, dctk.maxlen, dctk.n_features)
        # create an array of zeros with shape x_dims
        # recall that python considers zeros and boolean FALSE as the same
        x_pred = np.zeros(x_dims)

        # create a seq vector for our randomly select sequence 
        # i.e. create a numerical encoding for each char in the sequence 
        for t, char in enumerate(sentence):
            # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
            x_pred[0, t, dctk.char_int[char]] = 1

        # next, take the seq vector and pass into model to get a prediction of what the next char should be 
        preds = model.predict(x_pred, verbose=0)[0]
        # use the sample helper function to get index for next char 
        next_index = sample(preds)
        # use look up dict to get next char 
        next_char = dctk.int_char[next_index]

        # append next char to sequence 
        sentence = sentence[1:] + next_char 
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [39]:
# need this for on_epoch_end()
text = " ".join(clean_sonnets)

In [40]:
# create callback object that will print out text generation at the end of each epoch 
# use for real-time monitoring of model performance
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

----
### Train Model

Build a text generation model using LSTMs. Feel free to reference the model used in the guided project. 

It is recommended that you train this model to at least 50 epochs (but more if you're computer can handle it). 

You are free to change up the architecture as you wish. 

Just in case you have difficultly training a model, there is a pre-trained model saved to a file called `trained_text_gen_model.h5` that you can load in (the same way that you learned how to load in Keras models in Sprint 2 Module 4). 

In [41]:
from tensorflow.keras.layers import Dropout

# build text generation model layer by layer 
# fit model

n_features = 27
maxlen = 41

lstm = Sequential()
lstm.add(LSTM(128, input_shape=(maxlen, n_features)))
lstm.add(Dropout(0.2))
lstm.add(Dense(27, activation="softmax"))

lstm.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

lstm.fit(
    X, y, batch_size=32,
    epochs=15
)

Epoch 1/15
681/681 [==============================] - 144s 204ms/step - loss: 2.8459 - accuracy: 0.1943
Epoch 2/15
681/681 [==============================] - 141s 206ms/step - loss: 2.3681 - accuracy: 0.3179
Epoch 3/15
681/681 [==============================] - 133s 196ms/step - loss: 2.2267 - accuracy: 0.3508
Epoch 4/15
681/681 [==============================] - 137s 201ms/step - loss: 2.1287 - accuracy: 0.3703
Epoch 5/15
681/681 [==============================] - 135s 198ms/step - loss: 2.0522 - accuracy: 0.3934
Epoch 6/15
681/681 [==============================] - 132s 194ms/step - loss: 1.9838 - accuracy: 0.4072
Epoch 7/15
681/681 [==============================] - 136s 199ms/step - loss: 1.9209 - accuracy: 0.4295
Epoch 8/15
681/681 [==============================] - 136s 199ms/step - loss: 1.8558 - accuracy: 0.4410
Epoch 9/15
681/681 [==============================] - 133s 196ms/step - loss: 1.8170 - accuracy: 0.4491
Epoch 10/15
681/681 [==============================] - 139s 204m

In [42]:
# save trained model to file 
lstm.save("trained_text_gen_model.h5")

### Let's play with our trained model 

Now that we have a trained model that, though far from perfect, is able to generate actual English words, we can take a look at the predictions to continue to learn more about how a text generation model works. 

We can also take this as an opportunity to unpack the `def on_epoch_end` function to better understand how it works. 

In [43]:
# this is our joinned clean sonnet data
text

'and so of you beauteous and lovely youth no more be grievd at that which thou hast done who moving others are themselves as stone o let me suffer being at your beck but you like none none you for constant heart to show false art what beauty was of yore spendst thou thy fury on some worthless song  eif an individual project gutenbergtm electronic work is posted some in their garments though newfangled ill nor think the bitterness of absence sour the guilty goddess of my harmful deeds  cthe project gutenberg literary archive foundation the foundation who with his fear is put beside his part whateer thy thoughts or thy hearts workings be cxi civ of bird of flower or shape which it doth latch not making worse what nature made so clear which hath not figurd to thee my true spirit mine eye my heart thy pictures sight would bar and yet thou wilt for i being pent in thee when other petty griefs have done their spite o know sweet love i always write of you but why thy odour matcheth not thy sh

In [44]:
# randomly pick a starting index 
# will be used to take a random sequence of chars from `text`
# run this cell a few times and you'll see `start_index` is random
start_index = random.randint(0, len(text) - dctk.maxlen - 1)
start_index

53440

In [45]:
# next use the randomly selected starting index to sample a sequence from the `text`

# this is our seed string (i.e. input seqeunce into the model)
generated = ""

# start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
sentence = text[start_index: start_index + dctk.maxlen]

# add to generated
generated += sentence

generated

'e course of altering things for term of l'

In [46]:
# this block of code let's us know what the seed string is 
# i.e. the input seqeunce into the model
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

----- Generating with seed: "e course of altering things for term of l"
e course of altering things for term of l

In [47]:
# use model to predict what the next 40 chars should be that follow the seed string
for i in range(40):

    # shape of a single sample in a rank 3 tensor 
    x_dims = (1, dctk.maxlen, dctk.n_features)
    # create an array of zeros with shape x_dims
    # recall that python considers zeros and boolean FALSE as the same
    x_pred = np.zeros(x_dims)

    # create a seq vector for our randomly select sequence 
    # i.e. create a numerical encoding for each char in the sequence 
    for t, char in enumerate(sentence):
        # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
        x_pred[0, t, dctk.char_int[char]] = 1

    # next, take the seq vector and pass into model to get a prediction of what the next char should be 
    preds = lstm.predict(x_pred, verbose=0)[0]
    # use the sample helper function to get index for next char 
    next_index = sample(preds)
    # use look up dict to get next char 
    next_char = dctk.int_char[next_index]

    # append next char to sequence 
    sentence = sentence[1:] + next_char 

In [48]:
# this is the seed string
generated

'e course of altering things for term of l'

In [49]:
# these are the 40 chars that the model thinks should come after the seed stirng
sentence

'luese within the praise hsuldes in my did'

In [50]:
# how put it all together
generated + sentence

'e course of altering things for term of lluese within the praise hsuldes in my did'

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN